In [1]:
%load_ext autoreload
%autoreload 2

from transformers import AutoTokenizer, AutoModel
from autoLRP.LRPTensor import LRPTensor
from autoLRP.LRPnn import LRPModel
import matplotlib.pyplot as plt

# Setup device
device = "mps"

# Load pre-trained model and tokenizer
model_name = "sentence-transformers/all-mpnet-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

# 
text = "A woman is playing the guitar."
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"].to(device)
fe = model.embeddings.word_embeddings(input_ids)
tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze().tolist())


/opt/miniconda3/envs/waleed/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'LRPLayer' from partially initialized module 'autoLRP.LRPnn' (most likely due to a circular import) (/Users/waleedalasad/Documents/GitHub/autoLRP/autoLRP/LRPnn.py)

In [ ]:
fe = LRPTensor(fe).to(device)
lrp_model = LRPModel(model).to(device)
he = lrp_model(inputs_embeds=fe).last_hidden_state.mean(dim=1)


In [3]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the smallest GPT2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

def encode_sentence(sentence):
    # Tokenize and encode the input sentence
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Generate embeddings
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    
    # Get the last hidden state (you can experiment with other layers)
    embeddings = outputs.hidden_states[-1]
    
    return embeddings, inputs.input_ids

def decode_embeddings(embeddings):
    # Use the language model head to generate logits
    logits = model.lm_head(embeddings)
    
    # Get the most likely token at each position
    generated_ids = torch.argmax(logits, dim=-1)
    
    # Decode the generated ids back to text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    
    return generated_text

def generate_response(embeddings):
    # Use the embeddings as the initial hidden state
    with torch.no_grad():
        outputs = model.generate(
            inputs_embeds=embeddings,
            max_length=100,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=50,
            top_p=0.95,
            temperature=0.7
        )
    
    # Decode the generated ids back to text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

# Example usage
sentence1 = "Is the following sentence plausible? 'LeBron James took a corner kick.'"
sentence2 = "Is the following sentence plausible? 'LeBron James took a corner kick.' I think the answer is plausible, but I'm curious to hear what you think."

# Encode sentences
embeddings1, input_ids1 = encode_sentence(sentence1)
embeddings2, input_ids2 = encode_sentence(sentence2)

# Decode embeddings back to sentences
decoded_sentence1 = decode_embeddings(embeddings1)
decoded_sentence2 = decode_embeddings(embeddings2)

# Generate responses based on embeddings
response1 = generate_response(embeddings1)
response2 = generate_response(embeddings2)

print("Original Sentence 1:", sentence1)
print("Decoded Sentence 1:", decoded_sentence1)
print("Generated Response 1:", response1)
print("\nOriginal Sentence 2:", sentence2)
print("Decoded Sentence 2:", decoded_sentence2)
print("Generated Response 2:", response2)

/opt/miniconda3/envs/waleed/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/miniconda3/envs/waleed/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/miniconda3/envs/waleed/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a c

Original Sentence 1: Is the following sentence plausible? 'LeBron James took a corner kick.'
Decoded Sentence 1:  the best? be?
Theaving James is a shot pass to

Generated Response 1: 

,
.


Original Sentence 2: Is the following sentence plausible? 'LeBron James took a corner kick.' I think the answer is plausible, but I'm curious to hear what you think.
Decoded Sentence 2:  the best? be?
Theaving James is a shot pass to
 don it answer is yes. but I don not to see what the think.

Generated Response 2: 

.
,
 (
-
 and the
 in the "
 ", the, " (, and, ( a, a
:
 I. " " in a. ( " and a and



—
 [



In [10]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Check if 'mps' is available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Load the smallest GPT2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

def encode_sentence(sentence):
    # Tokenize and encode the input sentence
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Generate embeddings
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    
    # Get the last hidden state (you can experiment with other layers)
    embeddings = outputs.hidden_states[-1]
    
    return embeddings, inputs['input_ids'], inputs['attention_mask']

def decode_embeddings(embeddings, attention_mask):
    # Use the language model head to generate logits
    logits = model.lm_head(embeddings)
    
    # Get the most likely token at each position
    generated_ids = torch.argmax(logits, dim=-1)
    
    # Decode the generated ids back to text
    generated_text = tokenizer.decode(generated_ids[0].cpu(), skip_special_tokens=True)
    
    return generated_text

def generate_response(embeddings, attention_mask):
    # Use the embeddings as the initial hidden state
    with torch.no_grad():
        outputs = model.generate(
            inputs_embeds=embeddings,
            attention_mask=attention_mask,
            max_length=100,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id
        )
    
    # Decode the generated ids back to text
    generated_text = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)
    
    return generated_text

# Example usage
sentence1 = "Is the following sentence plausible? 'LeBron James took a corner kick.'"
sentence2 = "Is the following sentence plausible? 'LeBron James took a corner kick.' I think the answer is plausible, but I'm curious to hear what you think."

# Encode sentences
embeddings1, input_ids1, attention_mask1 = encode_sentence(sentence1)
embeddings2, input_ids2, attention_mask2 = encode_sentence(sentence2)

# Decode embeddings back to sentences
decoded_sentence1 = decode_embeddings(embeddings1, attention_mask1)
decoded_sentence2 = decode_embeddings(embeddings2, attention_mask2)

# Generate responses based on embeddings
response1 = generate_response(embeddings1, attention_mask1)
response2 = generate_response(embeddings2, attention_mask2)

print("Original Sentence 1:", sentence1)
print("Decoded Sentence 1:", decoded_sentence1)
print("Generated Response 1:", response1)
print("\nOriginal Sentence 2:", sentence2)
print("Decoded Sentence 2:", decoded_sentence2)
print("Generated Response 2:", response2)


Original Sentence 1: Is the following sentence plausible? 'LeBron James took a corner kick.'
Decoded Sentence 1:  the best? be?
Theaving James is a shot pass to


Original Sentence 2: Is the following sentence plausible? 'LeBron James took a corner kick.' I think the answer is plausible, but I'm curious to hear what you think.
Decoded Sentence 2:  the best? be?
Theaving James is a shot pass to
 don it answer is yes. but I don not to see what the think.



In [14]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# Load the BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)

def encode_sentence(sentence):
    # Tokenize and encode the input sentence
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Generate embeddings
    with torch.no_grad():
        outputs = bert_model(**inputs)
    
    # Get the last hidden state
    embeddings = outputs.last_hidden_state
    return embeddings, inputs.input_ids

def decode_embeddings(embeddings):
    # Use the language model head to generate logits
    logits = bert_mlm.cls(embeddings)
    
    # Get the most likely token at each position
    generated_ids = torch.argmax(logits, dim=-1)
    
    # Decode the generated ids back to text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return generated_text

def generate_response(embeddings):
    # Use the embeddings to predict the next tokens
    with torch.no_grad():
        logits = bert_mlm.cls(embeddings)
    
    # Generate text using a simple greedy decoding strategy
    generated_ids = []
    for _ in range(50):  # Generate up to 50 tokens
        next_token_logits = logits[0, -1, :]
        next_token = torch.argmax(next_token_logits).unsqueeze(0)
        generated_ids.append(next_token.item())
        
        # Update embeddings with the new token
        new_embeddings = bert_model(input_ids=next_token.unsqueeze(0)).last_hidden_state
        embeddings = torch.cat([embeddings, new_embeddings], dim=1)
        logits = bert_mlm.cls(embeddings)
    
    # Decode the generated ids back to text
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    return generated_text

# Example usage
sentence1 = "Is the following sentence plausible? 'LeBron James took a corner kick.'"
sentence2 = "Is the following sentence plausible? 'LeBron James took a corner kick.' I think the answer is plausible, but I'm curious to hear what you think."

# Encode sentences
embeddings1, input_ids1 = encode_sentence(sentence1)
embeddings2, input_ids2 = encode_sentence(sentence2)

# Decode embeddings back to sentences
decoded_sentence1 = decode_embeddings(embeddings1)
decoded_sentence2 = decode_embeddings(embeddings2)

# Generate responses based on embeddings
response1 = generate_response(embeddings1)
response2 = generate_response(embeddings2)

print("Original Sentence 1:", sentence1)
print("Decoded Sentence 1:", decoded_sentence1)
print("Generated Response 1:", response1)
print("\nOriginal Sentence 2:", sentence2)
print("Decoded Sentence 2:", decoded_sentence2)
print("Generated Response 2:", response2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Original Sentence 1: Is the following sentence plausible? 'LeBron James took a corner kick.'
Decoded Sentence 1: . is the following sentence plausible?'lebron james took a'kick. '.
Generated Response 1: ., often many, often many, often many, often many, often many, often many, often many, often many, often many, often many, often many, often many, often many, often many, often many, often many,

Original Sentence 2: Is the following sentence plausible? 'LeBron James took a corner kick.' I think the answer is plausible, but I'm curious to hear what you think.
Decoded Sentence 2: . is the following sentence plausible?'lebron james took a corner kick.'i think the answer is plausible, but i. m curious to hear what you think..
Generated Response 2: ., often many, often many, often many, often many, often many, often many, often many, often many, often many, often many, often many, often many, often many, often many, often many, often many,


In [1]:
from transformers import AutoTokenizer
import transformers 
import torch
model = "TinyLlama/TinyLlama_v1.1"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="mps",
)

sequences = pipeline(
    'The TinyLlama project aims to pretrain a 1.1B Llama model on 3 trillion tokens. With some proper optimization, we can achieve this within a span of "just" 90 days using 16 A100-40G GPUs 🚀🚀. The training has started on 2023-09-01.',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    repetition_penalty=1.5,
    eos_token_id=tokenizer.eos_token_id,
    max_length=500,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

/opt/miniconda3/envs/waleed/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Result: The TinyLlama project aims to pretrain a 1.1B Llama model on 3 trillion tokens. With some proper optimization, we can achieve this within a span of "just" 90 days using 16 A100-40G GPUs 🚀🚀. The training has started on 2023-09-01.
A few words about the Llamas: they come with some interesting characteristics (the most notable is that there are more females than males). This means that you need at least two male and one female model in each layer for the same task — so it'll have an average size similar if not slightly bigger per layers compared when you use all models together as single one. For example, our 5*5 TINKER model had an averge of around ~7M parameters while having only four of them per layer instead of eight like other TRIP-based architectures would do...
The pretrained weights were also optimized by running multiple reranking/repackaging experiments before finally converging towards those which yielded usable results; these are detailed below!
